In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy
from langdetect import detect

In [2]:
#read in data
df = pd.read_csv("movies_dataset/movies.csv")

In [3]:
#Filter data from 2010 - 2016 (AO3 start date and end date of movie data
df["year"] = df["year"].loc[df["year"] >= 2010]

In [4]:
# drop NA
df = df.dropna()
#df

In [5]:
#set column types
df["name"] = df["name"].astype(object)
df["year"] = df["year"].astype(int)
df.dtypes

name         object
rating       object
genre        object
year          int32
released     object
score       float64
votes       float64
director     object
writer       object
star         object
country      object
budget      float64
gross       float64
company      object
runtime     float64
dtype: object

In [6]:
# Create and format movie_tag column to match data from AO3
df["movie_tag"] = df["name"] + " (" + df["year"].astype("string") + ")"
#df

In [7]:
# Save clean table
df.to_csv("movie_clean.csv")

#Imports for scraping datta
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
# Scrape site and Parse the HTML
url = 'https://archiveofourown.org/media/Movies/fandoms'
browser.visit(url)
html = browser.html
html_soup = soup(html, 'html.parser')
# Scrape the number of fics written
title = []
tags_group = html_soup.find('ol', class_='alphabet fandom index group')
tags = tags_group.find_all("a", class_="tag")
amount = tags_group.find_all("li")
#print(tags_group)
for tag in tags:
    print(tag.text)
    title.append(tag.text)
    
    
# AO3 data list
list_ = []
for i in amount:
    print(i.text)
    list_.append(i.text)
    
    
# Scrape AO3
lists = []
tags = html_soup.find_all('ul', class_= "tags index group")
for tag in tags:
    print(tag.text)
    list_.append(tag.text)

browser.quit()

# Creating DataFrame for A03 
tag_df = pd.DataFrame(list_, columns=["movie_tag"])

# Save data
tag_df.to_csv("movies_dataset/ao3_dirty.csv")

In [8]:
#read in data
tag_df = pd.read_csv("movies_dataset/ao3_dirty.csv")
tag_df

,Unnamed: 0,movie_tag
0,0,\n\n 0\n \n↑\n\n\n\n\n00 Sch...
1,1,\n00 Schneider - Jagd auf Nihil Baxter (1994)\...
2,2,\n\n 1\n \n↑\n\n\n\n\n1 Per ...
3,3,\n1 Per Cent | Outlaws (Australia Movie 2017)\...
4,4,\n10 Cloverfield Lane (2016)\n ...
...,...,...
10300,10300,\n\nV for Vendetta - All Media Types\n ...
10301,10301,\n\nW pustyni i w puszczy | In Desert and Wild...
10302,10302,\n\nX (Movie 2022)\n (9)\n ...
10303,10303,\n\nY tu mamá también (2001)\n ...


In [9]:
#clean data
tag_df = tag_df.drop(columns=["Unnamed: 0"])
tag_df["movie_tag"] = tag_df["movie_tag"].str.replace("\n", "")
tag_df = tag_df["movie_tag"].str.rsplit(n=1, expand=True)
tag_df.rename(columns={0:'movie_tag', 1:'fan_fic'}, inplace=True)
tag_df["fan_fic"] = tag_df["fan_fic"].str.strip("("")")
tag_df["fan_fic"] = tag_df["fan_fic"].astype(int)
tag_df.head()

,movie_tag,fan_fic
0,0 ↑00 Schneider - Jagd auf ...,1
1,00 Schneider - Jagd auf Nihil Baxter (1994),1
2,1 ↑1 Per Cent | Outlaws (Au...,2
3,1 Per Cent | Outlaws (Australia Movie 2017),2
4,10 Cloverfield Lane (2016),13


In [10]:
tag_df.to_csv("tag_clean.csv")

In [12]:
tag_df = tag_df[tag_df["movie_tag"].apply(detect).eq('en')]


LangDetectException: No features in text.